In [21]:
#code from https://github.com/mswerli/zillow_data
import requests
import pandas as pd
from xml.etree import ElementTree
import xmltodict
from bs4 import BeautifulSoup
import numpy as np
import urllib
import utils
import datetime
import csv
import os.path
import json
from pprint import pprint
from config import Zillow_API_key

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy


#os.path.isfile(fname)

In [22]:
##Tags to extract from respons##
search_tags = (('address'),
            ('zestimate','valuationRange','high'),
            ('zestimate','valuationRange','low'),
            ('zestimate', 'amount'))

comp_tags =  (('address'),
          ('zestimate','valuationRange','high'),
          ('zestimate','valuationRange','low'),
          ('zestimate', 'amount'))

##Colums for data##
home_cols = ['street', 'zipcode', 'city', 'state',
         'latitude', 'longitude', 'currency1',
         'valuation_high',  'currency2', 'valuation_low',
         'currency3' ,'zestimate', 'zpid']

comp_cols = ['city', 'latitude', 'longitude', 'state',
         'street', 'zipcode',  'valuation_high',
         'currency1', 'valuation_low', 'currency2',
         'zestimate', 'currency3', 'zpid']

In [23]:
schoolRanking_medianPrice_csv = "datasets/schoolRanking_medianPrice.csv"
schoolRanking_medianPrice_df = pd.read_csv(schoolRanking_medianPrice_csv)
schoolRanking_medianPrice_df.head()

,Unnamed: 0,Name,Ranking,Address,zipcode,median,latitude,longitude,HomeAddress,CityStateZip
0,0,Canyon Crest Academy,1,"5951 Village Center Loop Rd.San Diego, CA 92130",92130,896100,32.944802,-117.199324,13014 Carita Cv,"San Diego, CA 92130"
1,1,Torrey Pines High School,2,"3710 Del Mar Heights Rd.San Diego, CA 92130",92130,896100,32.944802,-117.199324,13014 Carita Cv,"San Diego, CA 92130"
2,2,Westview High School,3,"13500 Camino Del SurSan Diego, CA 92129",92129,652300,32.966089,-117.127301,9090 Oviedo St,"San Diego, CA 92129"
3,3,Del Norte High School,4,"16601 Nighthawk Ln.San Diego, CA 92127",92127,683300,33.022206,-117.126479,16548 Nicole Ridge Rd,"San Diego, CA 92127"
4,4,Coronado High School,5,"650 D Ave.Coronado, CA 92118",92118,1424100,32.645759,-117.160716,710 E Ave,"Coronado, CA 92118"


In [24]:
# Get comparables for each zipcode
zipcode_zpids = {}
prices = {}
for i, row in schoolRanking_medianPrice_df.iterrows():
    zipcode = str(row.zipcode)
    zipcode_zpids[zipcode] = []
    zws_id = Zillow_API_key
    school =  row.Name
    search_params = {
        "zws_id": zws_id,
        "address": row.HomeAddress,
        "citystatezip": row.CityStateZip
    }
   
    r = utils.get_response(api = 'search', params = search_params)
    #print(r)
    details = utils.parse_response(response = r,
                                     api = 'search',
                                     tags = search_tags,
                                     cols = home_cols)
    zpid = details.zpid[0]
    
    search_params = {
        "zpid": zpid,
        "zws_id": zws_id,
        "count": "20"
    }
    
    r = utils.get_response(api = 'comp', params = search_params)
    comps = utils.parse_response(response = r,
                                     api = 'comp',
                                     tags = comp_tags,
                                     cols = comp_cols)
    
    #print(comps)
    for i, row in comps.iterrows():
   
        prices[str(row['zpid'])] = row['zestimate']
    
    
    zipcode_zpids[zipcode].append(comps['zpid'].tolist())
    
    #print (details)
#print (zipcode_zpids)



Starting....
Search API
http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&address=13014 Carita Cv&citystatezip=San Diego, CA 92130
Parsing desired data from response.....
address
('zestimate', 'valuationRange', 'high')
('zestimate', 'valuationRange', 'low')
('zestimate', 'amount')
Combining data frames
Setting column names
Starting....
http://www.zillow.com/webservice/GetComps.htm?
http://www.zillow.com/webservice/GetComps.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=66111761&count=20
http://www.zillow.com/webservice/GetComps.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=66111761&count=20
Parsing desired data from response.....
address
('zestimate', 'valuationRange', 'high')
('zestimate', 'valuationRange', 'low')
('zestimate', 'amount')
zpid: ['66111807', '66111861', '66111937', '66111769', '247631235', '248981029', '249295363', '249829384', '249980231', '250191926', '250192076', '250192123', '250192209', '250192559', '250192756', '250192759', '250192801', '

('zestimate', 'valuationRange', 'high')
('zestimate', 'valuationRange', 'low')
('zestimate', 'amount')
Combining data frames
Setting column names
Starting....
http://www.zillow.com/webservice/GetComps.htm?
http://www.zillow.com/webservice/GetComps.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=16745833&count=20
http://www.zillow.com/webservice/GetComps.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=16745833&count=20
Parsing desired data from response.....
address
('zestimate', 'valuationRange', 'high')
('zestimate', 'valuationRange', 'low')
('zestimate', 'amount')
zpid: ['16746117', '16743314', '16743442', '16743690', '16743696', '16744403', '16744604', '16744610', '16744649', '16744687', '16744712', '16744739', '16744761', '16744778', '16744828', '16745646', '16745677', '16745681', '16745706', '16745738']
Combining data frames
Setting column names
Starting....
Search API
http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&address=2817 Caminito Turnberry&citystat

('zestimate', 'valuationRange', 'high')
('zestimate', 'valuationRange', 'low')
('zestimate', 'amount')
Combining data frames
Setting column names
Starting....
http://www.zillow.com/webservice/GetComps.htm?
http://www.zillow.com/webservice/GetComps.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=16699707&count=20
http://www.zillow.com/webservice/GetComps.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=16699707&count=20
Parsing desired data from response.....
address
('zestimate', 'valuationRange', 'high')
('zestimate', 'valuationRange', 'low')
('zestimate', 'amount')
zpid: ['16693081', '16692732', '16692808', '16692813', '16692917', '16692918', '16692919', '16693086', '16693087', '16693099', '16693112', '16693232', '16693236', '16693277', '16693283', '16693285', '16693298', '16693373', '16693394', '16693408']
Combining data frames
Setting column names
Starting....
Search API
http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&address=13976 Country Creek Rd&citystate

Parsing desired data from response.....
address
('zestimate', 'valuationRange', 'high')
('zestimate', 'valuationRange', 'low')
('zestimate', 'amount')
Combining data frames
Setting column names
Starting....
http://www.zillow.com/webservice/GetComps.htm?
http://www.zillow.com/webservice/GetComps.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=60322827&count=20
http://www.zillow.com/webservice/GetComps.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=60322827&count=20
Parsing desired data from response.....
address
('zestimate', 'valuationRange', 'high')
('zestimate', 'valuationRange', 'low')
('zestimate', 'amount')
zpid: ['17146440', '17185863', '17186244', '17186273', '17186556', '17186594', '17186599', '17186601', '21698602', '64453727', '52523525', '48181470', '49942282', '49942294', '50205163', '50205166', '51081437', '51081513', '51081553', '51081591']
Combining data frames
Setting column names


In [25]:
with open('datasets/zipcode_zpids.json', 'w') as fp:
    json.dump(zipcode_zpids, fp)

In [26]:
zipcode_zpids = None
with open('datasets/zipcode_zpids.json') as zpid_json_data:
    zipcode_zpids = json.load(zpid_json_data)
    pprint(zipcode_zpids)

{'91913': [['250194558',
            '241609853',
            '17162327',
            '17185349',
            '17185350',
            '17185393',
            '17185430',
            '17185433',
            '121039872',
            '102972211',
            '52523235',
            '52523236',
            '54053250',
            '54483659',
            '54484159',
            '54704847',
            '54907639',
            '54907663',
            '54907669',
            '54907686']],
 '91915': [['17146440',
            '17185863',
            '17186244',
            '17186273',
            '17186556',
            '17186594',
            '17186599',
            '17186601',
            '21698602',
            '64453727',
            '52523525',
            '48181470',
            '49942282',
            '49942294',
            '50205163',
            '50205166',
            '51081437',
            '51081513',
            '51081553',
            '51081591']],
 '91941': [['17034149',
        

In [27]:
details_list = {}
idx = 1
for i, row in schoolRanking_medianPrice_df.iterrows():
    zipcode = row.zipcode
    school =  row.Name
       
    ##User specific ID used for IDing requests
    zws_id = Zillow_API_key
      
    zpids = zipcode_zpids[str(zipcode)][0]
    print(zpids)
    
    
    for zpid in zpids:
        try:
            search_params = {
                "zpid": zpid,
                "zws_id": zws_id
            }

            r = utils.get_response(api = 'propertyDetails', params = search_params)
            print(r)
            cols = ['useCode','bedrooms','bathrooms','finishedSqFt','lotSizeSqFt','yearBuilt','numFloors','numRooms','parkingType']
            details = utils.parse_response(response = r,
                                         api = 'propertyDetails',
                                         tags = (),
                                         cols = cols)
            details['school'] = school
            details['id'] = idx
            details['price'] = prices[str(zpid)]
            idx += 1
            print(details)
            details_list[idx]=details
        except:
            print(f'Could not process zpid {zpid}')
    
print(details_list)   


['66111807', '66111861', '66111937', '66111769', '247631235', '248981029', '249295363', '249829384', '249980231', '250191926', '250192076', '250192123', '250192209', '250192559', '250192756', '250192759', '250192801', '250192805', '250192861', '250192898']
Starting....
http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=66111807
<Response [200]>
Parsing desired data from response.....
{'zpid': '66111807', 'street': '6199 Verda Ln', 'zipcode': '92130', 'city': 'SAN DIEGO', 'state': 'CA', 'latitude': '32.962252', 'longitude': '-117.183473', 'image': 'https://photos.zillowstatic.com/p_d/ISmyocsmsyxfgh1000000000.jpg', 'useCode': 'Townhouse', 'bedrooms': '3', 'bathrooms': '2.5', 'finishedSqFt': '1478', 'lotSizeSqFt': '155558', 'yearBuilt': '2006', 'numFloors': None, 'numRooms': None, 'parkingType': None, 'school': 'Canyon Crest Academy', 'id': 1, 'price': '755293'}
Starting....
http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-

<Response [200]>
Parsing desired data from response.....
{'zpid': '66111861', 'street': '6139 Verda Ln', 'zipcode': '92130', 'city': 'SAN DIEGO', 'state': 'CA', 'latitude': '32.960973', 'longitude': '-117.184116', 'image': 'https://photos.zillowstatic.com/p_d/ISq5p4pjs27w680000000000.jpg', 'useCode': 'Townhouse', 'bedrooms': '3', 'bathrooms': '3.0', 'finishedSqFt': '1563', 'lotSizeSqFt': '135374', 'yearBuilt': '2006', 'numFloors': '2', 'numRooms': '7', 'parkingType': 'Garage - Attached', 'school': 'Torrey Pines High School', 'id': 10, 'price': '805692'}
Starting....
http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=66111937
<Response [200]>
Parsing desired data from response.....
{'zpid': '66111937', 'street': '6126 Mystra Pt', 'zipcode': '92130', 'city': 'San Diego', 'state': 'CA', 'latitude': '32.961402', 'longitude': '-117.185307', 'image': 'https://photos.zillowstatic.com/p_d/IS6yxeuvyguooz0000000000.jpg', 'useCode': 'SingleFamily', 

<Response [200]>
Parsing desired data from response.....
Could not process zpid 16805470
Starting....
http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=16805542
<Response [200]>
Parsing desired data from response.....
{'zpid': '16805542', 'street': '12719 Mengibar Ave', 'zipcode': '92129', 'city': 'SAN DIEGO', 'state': 'CA', 'latitude': '32.948264', 'longitude': '-117.110634', 'image': 'https://photos.zillowstatic.com/p_d/IS1znwkqc5rto31000000000.jpg', 'useCode': 'SingleFamily', 'bedrooms': '4', 'bathrooms': '3.0', 'finishedSqFt': '2304', 'lotSizeSqFt': '7501', 'yearBuilt': '1977', 'numFloors': '2', 'numRooms': '11', 'parkingType': 'Garage - Attached, Off-street', 'school': 'Westview High School', 'id': 19, 'price': '819625'}
Starting....
http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=16805611
<Response [200]>
Parsing desired data from response.....
{'zpid': '16805611', 'street': '12546

<Response [200]>
Parsing desired data from response.....
{'zpid': '16810257', 'street': '9514 Hito Ct', 'zipcode': '92129', 'city': 'SAN DIEGO', 'state': 'CA', 'latitude': '32.941781', 'longitude': '-117.119207', 'image': 'https://photos.zillowstatic.com/p_d/ISe0xlv105od4s0000000000.jpg', 'useCode': 'SingleFamily', 'bedrooms': '4', 'bathrooms': '3.0', 'finishedSqFt': '2356', 'lotSizeSqFt': '7501', 'yearBuilt': '1993', 'numFloors': '2', 'numRooms': '9', 'parkingType': 'Garage - Attached', 'school': 'Westview High School', 'id': 31, 'price': '862105'}
Starting....
http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=16810267
<Response [200]>
Parsing desired data from response.....
{'zpid': '16810267', 'street': '9535 Hito Ct', 'zipcode': '92129', 'city': 'SAN DIEGO', 'state': 'CA', 'latitude': '32.94166', 'longitude': '-117.118475', 'image': 'https://photos.zillowstatic.com/p_d/ISewdkuznds5171000000000.jpg', 'useCode': 'SingleFamily', 'bedroo

<Response [200]>
Parsing desired data from response.....
{'zpid': '119907381', 'street': '15375 Tanner Ridge Cir', 'zipcode': '92127', 'city': 'SAN DIEGO', 'state': 'CA', 'latitude': '33.009576', 'longitude': '-117.135934', 'image': 'https://photos.zillowstatic.com/p_d/ISmmj0q43u2qdo1000000000.jpg', 'useCode': 'SingleFamily', 'bedrooms': '5', 'bathrooms': '6.0', 'finishedSqFt': '3799', 'lotSizeSqFt': '7501', 'yearBuilt': '2015', 'numFloors': '2', 'numRooms': '15', 'parkingType': 'Garage - Attached', 'school': 'Del Norte High School', 'id': 44, 'price': '1417501'}
Starting....
http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=119977303
<Response [200]>
Parsing desired data from response.....
{'zpid': '119977303', 'street': '15941 Babcock St', 'zipcode': '92127', 'city': 'SAN DIEGO', 'state': 'CA', 'latitude': '33.017995', 'longitude': '-117.131092', 'image': 'https://photos.zillowstatic.com/p_d/ISa1repl5w2ygl1000000000.jpg', 'useCode': 'S

...

<Response [200]>
Parsing desired data from response.....
{'zpid': '16838563', 'street': '8433 Prestwick Dr', 'zipcode': '92037', 'city': 'LA JOLLA', 'state': 'CA', 'latitude': '32.859955', 'longitude': '-117.247115', 'image': 'https://photos.zillowstatic.com/p_d/IS2znsdmdex0qv1000000000.jpg', 'useCode': 'SingleFamily', 'bedrooms': '5', 'bathrooms': '3.0', 'finishedSqFt': '3870', 'lotSizeSqFt': '20037', 'yearBuilt': '1970', 'numFloors': '2', 'numRooms': '12', 'parkingType': 'Garage - Attached', 'school': 'La Jolla High School', 'id': 397, 'price': '2536272'}
Starting....
http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=16838948
<Response [200]>
Parsing desired data from response.....
{'zpid': '16838948', 'street': '2416 Avenida De La Playa', 'zipcode': '92037', 'city': 'LA JOLLA', 'state': 'CA', 'latitude': '32.854402', 'longitude': '-117.251232', 'image': None, 'useCode': 'SingleFamily', 'bedrooms': '4', 'bathrooms': '5.0', 'finishedSqF

<Response [200]>
Parsing desired data from response.....
Could not process zpid 16840238
Starting....
http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=16848192
<Response [200]>
Parsing desired data from response.....
{'zpid': '16848192', 'street': '1466 Torrey Pines Rd', 'zipcode': '92037', 'city': 'LA JOLLA', 'state': 'CA', 'latitude': '32.847118', 'longitude': '-117.267852', 'image': 'https://photos.zillowstatic.com/p_d/ISifjolgxo3esa1000000000.jpg', 'useCode': 'SingleFamily', 'bedrooms': '4', 'bathrooms': '3.0', 'finishedSqFt': '3012', 'lotSizeSqFt': '10890', 'yearBuilt': '1953', 'numFloors': '2', 'numRooms': None, 'parkingType': 'Garage - Attached', 'school': 'La Jolla High School', 'id': 410, 'price': '2233336'}
Starting....
http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=16848223
<Response [200]>
Parsing desired data from response.....
{'zpid': '16848223', 'street': '7709 Prospect

<Response [200]>
Parsing desired data from response.....
{'zpid': '16898311', 'street': '8710 Los Coches Rd', 'zipcode': '92040', 'city': 'LAKESIDE', 'state': 'CA', 'latitude': '32.834204', 'longitude': '-116.90693', 'image': None, 'useCode': 'SingleFamily', 'bedrooms': '3', 'bathrooms': '2.0', 'finishedSqFt': '1428', 'lotSizeSqFt': '20908', 'yearBuilt': '1930', 'numFloors': '1', 'numRooms': None, 'parkingType': None, 'school': 'River Valley Charter School', 'id': 423, 'price': '433309'}
Starting....
http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=16898314
<Response [200]>
Parsing desired data from response.....
Could not process zpid 16898314
Starting....
http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=16898653
<Response [200]>
Parsing desired data from response.....
{'zpid': '16898653', 'street': '8759 Via Diego Ter', 'zipcode': '92040', 'city': 'LAKESIDE', 'state': 'CA', 'latitude':

<Response [200]>
Parsing desired data from response.....
{'zpid': '17186556', 'street': '1500 Ashford Castle Dr', 'zipcode': '91915', 'city': 'CHULA VISTA', 'state': 'CA', 'latitude': '32.630589', 'longitude': '-116.951948', 'image': 'https://photos.zillowstatic.com/p_d/ISmycrredh1hr91000000000.jpg', 'useCode': 'SingleFamily', 'bedrooms': '3', 'bathrooms': '2.5', 'finishedSqFt': '1448', 'lotSizeSqFt': '3920', 'yearBuilt': '2000', 'numFloors': '2', 'numRooms': None, 'parkingType': 'Garage - Attached', 'school': 'Eastlake High School', 'id': 435, 'price': '535813'}
Starting....
http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=17186594
<Response [200]>
Parsing desired data from response.....
{'zpid': '17186594', 'street': '1467 Ashford Castle Dr', 'zipcode': '91915', 'city': 'CHULA VISTA', 'state': 'CA', 'latitude': '32.630223', 'longitude': '-116.953258', 'image': 'https://photos.zillowstatic.com/p_d/ISynw4bl6djl370000000000.jpg', 'useCod

<Response [200]>
Parsing desired data from response.....
{'zpid': '51081437', 'street': '2549 Noble Canyon Rd', 'zipcode': '91915', 'city': 'CHULA VISTA', 'state': 'CA', 'latitude': '32.640904', 'longitude': '-116.952026', 'image': 'https://photos.zillowstatic.com/p_d/IS6yhia6b40r6f1000000000.jpg', 'useCode': 'SingleFamily', 'bedrooms': '4', 'bathrooms': '2.5', 'finishedSqFt': '2158', 'lotSizeSqFt': '7501', 'yearBuilt': '2002', 'numFloors': '2', 'numRooms': None, 'parkingType': 'Garage - Attached', 'school': 'Eastlake High School', 'id': 447, 'price': '623425'}
Starting....
http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=X1-ZWz183nrq3tjwr_a9fju&zpid=51081513
<Response [200]>
Parsing desired data from response.....
{'zpid': '51081513', 'street': '1359 Granite Springs Dr', 'zipcode': '91915', 'city': 'CHULA VISTA', 'state': 'CA', 'latitude': '32.637334', 'longitude': '-116.948991', 'image': 'https://photos.zillowstatic.com/p_d/ISuoekajo8bqvi0000000000.jpg', 'useCode

In [28]:
with open('datasets/all_comps.json', 'w') as fp:
    json.dump(details_list, fp)

In [29]:
all_comps_data = []
for key, value in details_list.items():
    all_comps_data.append(value)
all_comps_df = pd.DataFrame(all_comps_data)
all_comps_df.head()

all_comps_df.to_csv("datasets/all_comps.csv", index=False)

### all_comps.csv is added to app.py to import data to database

In [ ]:
# ##User specific ID used for IDing requests
# zws_id = Zillow_API_key
# ##Address of property##
# address = '870 avenida abeja'
# ##This one is intuitive
# citystatezip = 'San Marcos,CA,92069'
# ##Zillow ID for property. Get from URL in browswer
# zpid = '63141450'
# ##How any results
# count = '25'
# school='Canyon Crest Academy'

In [ ]:
#     ##Identifying info to base search on##
#     search_params = {'zws_id': zws_id, 'address': address,
#                     'citystatezip': citystatezip,
#                     'zpid': zpid, 'count': count}

In [ ]:
# ##Tags to extract from respons##
# search_tags = (('address'),
#                 ('zestimate','valuationRange','high'),
#                 ('zestimate','valuationRange','low'),
#                 ('zestimate', 'amount'))

# comp_tags =  (('address'),
#               ('zestimate','valuationRange','high'),
#               ('zestimate','valuationRange','low'),
#               ('zestimate', 'amount'))

In [ ]:
# ##Colums for data##
# home_cols = ['street', 'zipcode', 'city', 'state',
#              'latitude', 'longitude', 'currency1',
#              'valuation_high',  'currency2', 'valuation_low',
#              'currency3' ,'zestimate', 'zpid']

# comp_cols = ['city', 'latitude', 'longitude', 'state',
#              'street', 'zipcode',  'valuation_high',
#              'currency1', 'valuation_low', 'currency2',
#              'zestimate', 'currency3', 'zpid']

In [ ]:
# ##Get starting home data##
# r = utils.get_response(api = 'search', params = search_params)

# home = utils.parse_response(response = r,
#                             api = 'search',
#                             tags = search_tags,
#                             cols = home_cols)

In [ ]:
# ##Get Comps for original property
# comp_response = utils.get_response(api = 'comp', params = search_params)

# comps = utils.parse_response(response = comp_response,
#                              api = 'comp',
#                              tags = comp_tags,
#                              cols = comp_cols)

In [ ]:
# ##Combine data and write to csv
# home = home[comp_cols]

# data_list = [home,comps]
# all_data = pd.concat(data_list)
# all_data['record_date'] = datetime.datetime.now()
# # path = 'testcomps.csv'
# # all_data.to_csv(path)

In [ ]:
# #all_data.to_csv(path)
# details_list = []
# for i, row in all_data.iterrows():
#     #print (row.zpid)
#     search_params = {
#         "zpid": row.zpid,
#         "zws_id": zws_id
#     }
    
#     r = utils.get_response(api = 'propertyDetails', params = search_params)
    
#     details = utils.parse_response(response = r,
#                              api = 'propertyDetails',
#                              tags = comp_tags,
#                              cols = comp_cols)
#     details['school'] = school
#     #print(details)
#     details_list.append(details)
# comps_df = pd.DataFrame(details_list)
# print(comps_df)
# comps_df.to_csv("datasets/schoolcomps/" + school + ".csv")

# all_data.keys()

# if os.path.isfile(path):
#     ##check for exising file
#     existing_data = pd.read_csv(path)
#     existing_data = existing_data.drop('Unnamed: 0', axis = 1)
#     data_list = [all_data, existing_data]

#     all_data = pd.concat(data_list, sort = 'FALSE')

# all_data.to_csv(path)